<a href="https://colab.research.google.com/github/sandeepjunaghare/llm/blob/main/cohere_ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# practical applications - modern search engines, eCommerce product recomendations, social media content moderations, email spam filtering, customer support conversational agents


In [2]:
! pip install cohere altair -q

In [3]:
import cohere
import numpy as np
import pandas as pd
import altair as alt

co = cohere.Client("EwcsPBLocPue4X4ZwYKNIMiUM1pKgLd43JMl7bZo")


In [10]:
#load the dataset to a dataframe
from google.colab import files
import io

uploaded = files.upload()

for fn in uploaded.keys():
  print('local uploaded file:  "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  df_orig = pd.read_csv(io.BytesIO(uploaded[fn]),names=['intent','query'])



#print('User uploaded file "{fn}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
#df_orig = pd.read_csv('/Users/sandeep/Dropbox/dev/projects/cohere-ai/data-sets/atis/atis_intents_train.csv',names=['intent','query'])


# Take a small sample for illustration purposes
sample_classes = ['atis_airfare', 'atis_airline', 'atis_ground_service']
df = df_orig.sample(frac=0.12, random_state=30)


# Remove unnecessary column




Saving atis_intents_train.csv to atis_intents_train (5).csv
local uploaded file:  "atis_intents_train (5).csv" with length 381136 bytes
